In [1]:
import pandas as pd
import numpy as np
from urllib.request import urlopen
import json
import plotly.express as px
import plotly.graph_objects as go

In [2]:
vaccines = pd.read_excel('2023-2024_School_Vaccination_Coverage_Levels_Kindergarten.xlsx',
                         sheet_name = 'Coverage by County')

vaccines.rename(columns={'DTP/DTaP/DT/Td': 'Tdap'}, inplace = True)

vaccine_columns = vaccines.columns.to_list()
vaccine_columns.remove('County')
vaccine_columns

['Tdap', 'Hepatitis A', 'Hepatitis B', 'MMR', 'Polio', 'Varicella']

In [3]:
for col in vaccine_columns:
    vaccines[col] = pd.to_numeric(vaccines[col], errors = 'coerce')

In [4]:
# get geographic data for county borders
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)

In [5]:
fips_mapping = pd.read_csv('state_and_county_fips_master.csv') 
  # from https://github.com/kjhealy/fips-codes/blob/master/state_and_county_fips_master.csvb

In [6]:
fips_mapping = fips_mapping[fips_mapping['state']=='TX'].copy()
fips_mapping['name'] = fips_mapping['name'].str.removesuffix(' County').str.lower()
fips_mapping['fips'] = fips_mapping['fips'].astype(str)

In [7]:
vaccines['county_lower'] = vaccines['County'].str.lower()

In [8]:
vaccines = vaccines.merge(fips_mapping, how = 'left', left_on = 'county_lower', right_on = 'name')

In [9]:
# used to think about lower bound for plotting with regards to the color gradient, decided on 80%
for col in vaccine_columns:
    print(vaccines[col].quantile(.025))

0.8261612021857924
0.8333333333333334
0.8638440860215054
0.8045454545454546
0.8230874316939891
0.7833333333333333


In [37]:
for col in vaccine_columns:
    fig = px.choropleth_map(vaccines, geojson=counties, locations='fips', color=col,
                            color_continuous_scale="rdylgn",
                            range_color=(0.8, 1),
                            map_style="carto-positron",
                            zoom=4.7, center={"lat": 31.5, "lon": -101},
                            opacity=0.8,
                            hover_name='County',
                            hover_data={'fips': False,
                                        col: ':.1%'})
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0},
                      autosize=True)
    
    #fig.write_html(col + '_map.html') # easier for viewing on my laptop

    # creates html that can be pasted into google sites
    with open(col + '_map.html', 'w') as f: 
        f.write(fig.to_html(include_plotlyjs='cdn'))